# Petri networks

Here are only some tests with the snakes package to use on next session's 2 work.

In [1]:
import snakes.plugins
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
snakes.plugins.load(["gv", 'labels'], "snakes.nets", "nets")
from nets import *

path = 'images/'

net = PetriNet("mynet")
net.add_place(Place('p1', [dot]))
net.add_place(Place('p2'))
net.add_place(Place('p3', [dot]))
net.add_transition(Transition('t'))
net.add_input('p1', 't', Value(dot))
net.add_input('p3', 't', Value(dot))
net.add_output('p2', 't', Value(dot))
net.draw(f'{path}mynet33.png')


Graph(['node_2', 'node_1', 'node_0', 'node_3'], [])

In [2]:
t = net.transition('t')
m = t.modes()
t.fire(m.pop())
net.draw(f'{path}mynet2.ps')

Graph(['node_2', 'node_1', 'node_0', 'node_3'], [])

In [3]:
class Subject:
    def __init__(self, name, year_conclusion):
        self.name = name
        self.year_conclusion = year_conclusion
    def __str__(self):
        return str(self.name)

class Aluno:
    def __init__(self, ident, subjects):
        self.ident = ident
        self.subjects = subjects
    def __str__(self):
        return str(self.ident)
    def __repr__(self):
        return str(self.ident)
        
student = Aluno('Paulo', Subject('calculo1', '2014.1'))
print(student)
classes = PetriNet('MyNet2')
p1 = Place('p1', [student, dot])
#p1.label('aaa')
classes.add_place(p1)
classes.add_place(Place('p2'))
classes.add_place(Place('p3', [dot, student]))
classes.place('p2')

classes.add_transition(Transition('t1'))
classes.add_input('p1', 't1', Value(student))


classes.add_input('p3', 't1', Value(student))



classes.add_output('p2', 't1', Value(student))



classes.draw(f'{path}mynet44.png')
t1 = classes.transition('t1')
print(t1)
m1 = t1.modes()
print(m1)
t1.fire(m1.pop())
print(m1)
graph = classes.draw(f'{path}mynet45.png' )


Paulo
t1
[Substitution()]
[]


# Applying Petri To Educational Problems

In this new section, we will explore how to model a grade of students in order to visualize in which subjects students get most stuck on, by studying the flow of students in calculus chairs of the Science and Technology course of Federal University of Rio Grande do Norte. We will first do for calculus students and later, if possible, to all the subjects on S&T's curriculum.

## Importing libs and students data

In [4]:
import snakes.plugins
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
snakes.plugins.load(["gv", 'labels'], "snakes.nets", "nets")
from nets import *
import pandas as pd

In [5]:
components = pd.read_csv('data/componentes-curriculares-presenciais.csv', sep=';')
components.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38276 entries, 0 to 38275
Data columns (total 29 columns):
id_componente               38276 non-null int64
tipo_componente             38276 non-null object
codigo                      38276 non-null object
nivel                       38276 non-null object
nome                        38275 non-null object
unidade_responsavel         38276 non-null object
ch_teorico                  38276 non-null int64
ch_pratico                  38276 non-null int64
ch_estagio                  38276 non-null int64
ch_total                    38276 non-null int64
ch_dedicada_docente         38276 non-null int64
ch_ead                      38276 non-null int64
cr_max_ead                  38276 non-null int64
permite_horario_flexivel    38276 non-null bool
qtd_unidades                30183 non-null float64
procedimentos_avaliacao     3207 non-null object
equivalencia                11115 non-null object
pre_requisito               9638 non-null object
co

In [6]:
students = pd.read_csv('data/df_alunos20151.csv')
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20334 entries, 0 to 20333
Data columns (total 25 columns):
Unnamed: 0                  20334 non-null int64
descricao                   20334 non-null object
discente                    20334 non-null object
id_curso                    20334 non-null float64
id_turma                    20334 non-null int64
media_final                 14640 non-null object
codigo_turma                20334 non-null object
id_componente_curricular    20334 non-null int64
ano                         20334 non-null int64
periodo                     20334 non-null int64
data_inicio                 20334 non-null object
data_fim                    20334 non-null object
data_consolidacao           20333 non-null object
agrupadora                  20334 non-null bool
id_turma_agrupadora         4674 non-null float64
id_componente               17445 non-null float64
codigo                      17445 non-null object
nome                        17445 non-null obj

In [8]:
#Counting unique students on the dataset
students['discente'].unique().shape

(714,)

In [11]:
students[students['descricao'] == 'CANCELADO'].shape

(2037, 25)

As we can see, there are 714 students who entered the Science and Technology course in the first semester of 2015. Unfortunately, this does not mean that all of this students will finish the course on expected time of two years or would quit. This will also affect our petri network model in the future.

The reason for using these students is because after up from this period, course's grade was changed, and students from previous semesters would affect in our analysis, because how subjects were teached on the previous curriculum could affect their performance in how subjects are teached from 2015 and beyond.

## Preparing Students Object

Now we are going to create a student's model tu use as the token color on the colored petri network. Also, we will clean even more the data so we have only students who have not canceled their registration of the course.

In [17]:
students_not_null = students.dropna(subset=['media_final'])
students_not_null.head()

,Unnamed: 0,descricao,discente,id_curso,id_turma,media_final,codigo_turma,id_componente_curricular,ano,periodo,...,id_componente,codigo,nome,unidade_responsavel,pre_requisito,co_requisito,ano_programa,periodo_programa,curso_componente,anoperiodo
0,720645,REPROVADO POR MÉDIA E POR FALTAS,6360ce81e659feee39452aa8cd111beb,10320810.0,1242589,"0,0",03,57587,2015,1,...,57587.0,ECT2103,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,NaN,NaN,2019.0,2.0,CIÊNCIAS E TECNOLOGIA,20151
1,720648,CANCELADO,e5930e3c2c83d80ef671058e553b96b9,10320810.0,1242589,"1,3",03,57587,2015,1,...,57587.0,ECT2103,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,NaN,NaN,2019.0,2.0,CIÊNCIAS E TECNOLOGIA,20151
2,720651,REPROVADO,dab92c51983ebd2459e35439f103f686,10320810.0,1242589,"0,7",03,57587,2015,1,...,57587.0,ECT2103,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,NaN,NaN,2019.0,2.0,CIÊNCIAS E TECNOLOGIA,20151
3,720654,REPROVADO,d258f7fb1f7e7afa8f7b25b295307c7a,10320810.0,1242589,"0,0",03,57587,2015,1,...,57587.0,ECT2103,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,NaN,NaN,2019.0,2.0,CIÊNCIAS E TECNOLOGIA,20151
4,720657,REPROVADO,23b37af4ddf3030c9eb44e153457ed60,10320810.0,1242589,"3,0",03,57587,2015,1,...,57587.0,ECT2103,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,NaN,NaN,2019.0,2.0,CIÊNCIAS E TECNOLOGIA,20151


In [23]:
students['descricao'].value_counts(ascending=False)

APROVADO POR NOTA                   5364
APROVADO                            4462
INDEFERIDO                          2799
REPROVADO                           2798
CANCELADO                           2037
REPROVADO POR MÉDIA E POR FALTAS    1274
TRANCADO                             635
DESISTENCIA                          469
EXCLUIDA                             442
REPROVADO POR NOTA                    40
REPROVADO POR FALTAS                   8
REPROVADO POR NOTA E FALTA             6
Name: descricao, dtype: int64

In [22]:
students_not_null['descricao'].value_counts(ascending=False)

APROVADO POR NOTA                   5364
APROVADO                            4462
REPROVADO                           2798
REPROVADO POR MÉDIA E POR FALTAS    1274
CANCELADO                            678
REPROVADO POR NOTA                    40
REPROVADO POR FALTAS                   8
REPROVADO POR NOTA E FALTA             6
TRANCADO                               6
EXCLUIDA                               3
DESISTENCIA                            1
Name: descricao, dtype: int64

In [ ]:
excluded = students_not_null['descricao'] != 'EXCLUIDA'
give_up = students_not_null['descricao'] != 'DESISTENCIA'
cancelled = students_not_null['descricao'] != 'CANCELADO'